# PREPROCESSING

Study of a dataset cotaining various information about 32135 games published on Steam gaming plataform.

| Columna | Descripción | Ejemplo |
| -- | - | - |
| publisher | Empresa publicadora del contenido | [Ubisoft,Dovetail Games - Trains,Degica] |
| genres | Genero del contenido | [Action, Adventure, Racing, Simulation, Strategy] |
| app_name | Nombre del contenido | [Warzone, Soundtrack, Puzzle Blocks] |
| title | Titulo del contenido | [The Dream Machine: Chapter 4 , Fate/EXTELLA - Sweet Room Dream, Fate/EXTELLA - Charming Bunny] |
| url | URL de publicación del contenido | http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/ |
| release_date | Fecha de lanzamiento | [2018-01-04] |
| tags | etiquetas de contenido | [Simulation, Indie, Action, Adventure, Funny, Open World, First-Person, Sandbox, Free to Play] |
| discount_price | precio de descuento | [22.66, 0.49, 0.69] |
| reviews_url | Reviews de contenido | http://steamcommunity.com/app/681550/reviews/?browsefilter=mostrecent&p=1 |
| specs | Especificaciones | [Multi-player, Co-op, Cross-Platform Multiplayer, Downloadable Content] |
| price | Precio del contenido | [4.99, 9.99, Free to Use, Free to Play] |
| early_access | acceso temprano | [False, True] |
| id | identificador unico de contenido | [761140, 643980, 670290] |
| developer | Desarrollador | [Kotoshiro, Secret Level SRL, Poolians.com] |
| sentiment | Análisis de sentimiento | [Mixed, Very Positive, Positive, 3 user reviews] |
| metascore | Score por metacritic | [80, 74, 77, 75] |



Import relevant libraries.

In [286]:
import pandas as pd
import numpy as np
import ast
import json

## Read Data

I tried to open the json file with pd.read_json but i could not do it because of it's format.

In [213]:
# pd.read_json()

That's why I decided to use "open" and then transform each row into a dictionary. With the list of all the rows I can create a DataFrame.

In [214]:
# Open file.
file = open('datasets/steam_games.json')

allNewLines = []

# Go through each line 
for line in file:
    # Create a dictionary of each line.
    myNewdict = ast.literal_eval(line)
    # Append to a list of all the lines in the file.
    allNewLines.append(myNewdict)

rawGamesDf = pd.DataFrame(allNewLines)

In [215]:
rawGamesDf.head(2)

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN


## Columns: url & reviews_url
We do not need any url for the analysis so we will drop them.

In [216]:
rawGamesDf.drop(columns=['url', 'reviews_url'], inplace=True)

In [217]:
rawGamesDf.head(2)

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN


---

## Duplicates

Analysis if there are duplicates

In [218]:
# Games with duplicate ids, dropna to avoid games with ids = NaN.
auxDf = rawGamesDf[rawGamesDf.duplicated(keep=False, subset='id')].dropna(subset='id')
auxDf

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore
13894,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",NaN,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games,Mostly Positive,86
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",NaN,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games,Mostly Positive,86


In [219]:
index_last_dup_to_remove = rawGamesDf[rawGamesDf.duplicated(keep='last', subset='id')].dropna(subset='id').index
# index_last_dup_to_remove = rawGamesDf[rawGamesDf.duplicated(keep='last', subset='id')].dropna(subset='id')
index_last_dup_to_remove

Index([13894], dtype='int64')

In [220]:
rawGamesDf.iloc[index_last_dup_to_remove]

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore
13894,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",NaN,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games,Mostly Positive,86


In [221]:
rawGamesDf.drop(index=index_last_dup_to_remove, inplace=True)

In [222]:
# Can't use this one yet because of the list in some columns.
# rawGamesDf.drop_duplicates(inplace=True)

---

## Endpoint 1: genero

Processing data to be prepared for the "genero" function, that expects a year and returns a dictionary with the top 5 genres from that year.

Since the data is static and won't change, the most optimal solution could be to prepare a data with evry year already processed so the API endpoint can consume it with the least processing in real time.

Example:
| Year | Genres |
| -- | - |
| 2018 | {'Indie': 125, 'Action': 75, 'Adventure': 74, 'Casual': 45, 'Simulation': 43} |
| 2016 | {'Indie': 4106, 'Action': 2544, 'Casual': 2141, 'Adventure': 2042, 'Simulation': 1666} |

For this case the only thing that matters to us is "genres" and "release_date" columns, so we can drop the other ones.


In [290]:
generoEndpointDf = rawGamesDf[['genres', 'release_date']]
generoEndpointDf.head()

,genres,release_date
0,"[Action, Casual, Indie, Simulation, Strategy]",2018-01-04
1,"[Free to Play, Indie, RPG, Strategy]",2018-01-04
2,"[Casual, Free to Play, Indie, Simulation, Sports]",2017-07-24
3,"[Action, Adventure, Casual]",2017-12-07
4,NaN,NaN


Now we can drop all the columns that have a NaN in any of the two columns because they will bring no information to the scope of the genero function.

In [291]:
generoEndpointDf.describe()

,genres,release_date
count,28851,30067
unique,883,3582
top,[Action],2012-10-16
freq,1879,100


In [292]:
generoEndpointDf.isna().sum()

genres          3283
release_date    2067
dtype: int64

In [293]:
generoEndpointDf = generoEndpointDf.dropna()

In [294]:
generoEndpointDf.isna().sum()

genres          0
release_date    0
dtype: int64

### Year

We only need year for this study so we will create a new column called year with that information. Extracting year with a pattern of four numbers toghether.

In [295]:
generoEndpointDf['release_date'].unique()

array(['2018-01-04', '2017-07-24', '2017-12-07', ..., '2004-03-01',
       'January 2018', '2018-10-01'], dtype=object)

In [296]:
# Pattern of regex to extract, 1XXX or 2XXX (valid years).
pattern = '([1-2][0-9][0-9][0-9])'
generoEndpointDf['year'] = generoEndpointDf['release_date'].str.extract(pattern, expand = True)

In [297]:
generoEndpointDf['year'].head()

0    2018
1    2018
2    2017
3    2017
5    2018
Name: year, dtype: object

In [298]:
generoEndpointDf['year'].describe()

count     28732
unique       39
top        2017
freq       9284
Name: year, dtype: object

In [299]:
generoEndpointDf['year'].isna().sum()

101

Drop "release_date" and drop NaNs from year.

In [300]:
generoEndpointDf = generoEndpointDf.drop(columns='release_date')

In [301]:
generoEndpointDf.shape

(28833, 2)

In [302]:
generoEndpointDf = generoEndpointDf.dropna()

In [303]:
generoEndpointDf.shape

(28732, 2)

Lastly we convert years into int and we can check the years so they are valid years and drop the invalids.

In [304]:
generoEndpointDf['year'] = pd.to_numeric(generoEndpointDf['year'])
generoEndpointDf['year'].info()

<class 'pandas.core.series.Series'>
Index: 28732 entries, 0 to 32133
Series name: year
Non-Null Count  Dtype
--------------  -----
28732 non-null  int64
dtypes: int64(1)
memory usage: 448.9 KB


In [306]:
generoEndpointDf[generoEndpointDf['year'] > 2023]

,genres,year
13428,"[Casual, Indie, Early Access]",2756


In [307]:
generoEndpointDf[generoEndpointDf['year'] < 1980]

,genres,year


In [308]:
generoEndpointDf = generoEndpointDf[generoEndpointDf['year'] <= 2023]

### Genre

Each game can have various genres, we can see this in the "genres" column because it is filled with lists.

For this reason we need to explode it so we can count all the genres separately.

In [309]:
genresCompleteList = generoEndpointDf.explode(column=['genres'])['genres'].unique()
print(genresCompleteList)
print(genresCompleteList.shape)

['Action' 'Casual' 'Indie' 'Simulation' 'Strategy' 'Free to Play' 'RPG'
 'Sports' 'Adventure' 'Racing' 'Massively Multiplayer' 'Early Access'
 'Animation &amp; Modeling' 'Video Production' 'Web Publishing'
 'Education' 'Software Training' 'Utilities' 'Design &amp; Illustration'
 'Audio Production' 'Photo Editing' 'Accounting']
(22,)


In [310]:
generoEndpointDf.shape

(28731, 2)

In [311]:
generoEndpointDf = generoEndpointDf.explode('genres')
generoEndpointDf.head()

,genres,year
0,Action,2018
0,Casual,2018
0,Indie,2018
0,Simulation,2018
0,Strategy,2018


In [312]:
generoEndpointDf.shape

(71212, 2)

Now we can make a Groupby to get all the genres of each year. THen unstack it and have a new Data Frame with the data we desire.

In [313]:
generoEndpointDf2 = generoEndpointDf.groupby(by=['year', 'genres'])['genres'].count()
generoEndpointDf2.tail()

year  genres   
2019  RPG          2
      Strategy     1
2021  Adventure    1
      Indie        1
      RPG          1
Name: genres, dtype: int64

In [314]:
generoEndpointDf3 = generoEndpointDf2.unstack(level=1, fill_value=0)
generoEndpointDf3.head()

genres,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,Free to Play,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
year,,,,,,,,,,,,,,,,,,,,,
1983,0,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1984,0,1,1,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1986,0,0,1,0,0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
1987,0,0,3,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0


In [336]:
# Now we have exactly what we need.
generoEndpointDict = generoEndpointDf3.to_dict(orient='index')
generoEndpointDict

{1983: {'Accounting': 0,
  'Action': 1,
  'Adventure': 1,
  'Animation &amp; Modeling': 0,
  'Audio Production': 0,
  'Casual': 1,
  'Design &amp; Illustration': 0,
  'Early Access': 0,
  'Education': 0,
  'Free to Play': 0,
  'Indie': 0,
  'Massively Multiplayer': 0,
  'Photo Editing': 0,
  'RPG': 0,
  'Racing': 0,
  'Simulation': 0,
  'Software Training': 0,
  'Sports': 0,
  'Strategy': 0,
  'Utilities': 0,
  'Video Production': 0,
  'Web Publishing': 0},
 1984: {'Accounting': 0,
  'Action': 1,
  'Adventure': 1,
  'Animation &amp; Modeling': 0,
  'Audio Production': 0,
  'Casual': 2,
  'Design &amp; Illustration': 0,
  'Early Access': 0,
  'Education': 0,
  'Free to Play': 0,
  'Indie': 1,
  'Massively Multiplayer': 0,
  'Photo Editing': 0,
  'RPG': 0,
  'Racing': 0,
  'Simulation': 0,
  'Software Training': 0,
  'Sports': 0,
  'Strategy': 0,
  'Utilities': 0,
  'Video Production': 0,
  'Web Publishing': 0},
 1985: {'Accounting': 0,
  'Action': 0,
  'Adventure': 0,
  'Animation &amp;

Sort by values.

In [339]:
generoEndpointDict_sorted = {}
for y in generoEndpointDict.keys():
    genres = generoEndpointDict[y]
    genres_sorted = {k: v for k, v in sorted(genres.items(), key=lambda item: item[1], reverse=True)}
    # print(genres_sorted)
    generoEndpointDict_sorted[y] = genres_sorted

generoEndpointDict_sorted

{1983: {'Action': 1,
  'Adventure': 1,
  'Casual': 1,
  'Accounting': 0,
  'Animation &amp; Modeling': 0,
  'Audio Production': 0,
  'Design &amp; Illustration': 0,
  'Early Access': 0,
  'Education': 0,
  'Free to Play': 0,
  'Indie': 0,
  'Massively Multiplayer': 0,
  'Photo Editing': 0,
  'RPG': 0,
  'Racing': 0,
  'Simulation': 0,
  'Software Training': 0,
  'Sports': 0,
  'Strategy': 0,
  'Utilities': 0,
  'Video Production': 0,
  'Web Publishing': 0},
 1984: {'Casual': 2,
  'Action': 1,
  'Adventure': 1,
  'Indie': 1,
  'Accounting': 0,
  'Animation &amp; Modeling': 0,
  'Audio Production': 0,
  'Design &amp; Illustration': 0,
  'Early Access': 0,
  'Education': 0,
  'Free to Play': 0,
  'Massively Multiplayer': 0,
  'Photo Editing': 0,
  'RPG': 0,
  'Racing': 0,
  'Simulation': 0,
  'Software Training': 0,
  'Sports': 0,
  'Strategy': 0,
  'Utilities': 0,
  'Video Production': 0,
  'Web Publishing': 0},
 1985: {'Simulation': 1,
  'Accounting': 0,
  'Action': 0,
  'Adventure': 0,

In [340]:
with open("datasets/steam_games_endpoint_1_genero.json", "w") as outfile:
    json.dump(generoEndpointDict_sorted, outfile)

**TEST**

In [350]:
def genero(Year: str): 
    """
    Se ingresa un año y devuelve un diccionario con los 5 géneros más vendidos en el orden correspondiente.
    """

    # Read data
    with open("datasets/steam_games_endpoint_1_genero.json") as json_file:
        genres_dict = json.load(json_file)
    # Since data is already sorted, get the first 5 as top 5.
    top5 = dict(list(genres_dict[Year].items())[0:5])
    return top5

genero('2015')

{'Indie': 2828,
 'Action': 2033,
 'Adventure': 1502,
 'Casual': 1217,
 'Simulation': 1173}

---

---

---

## CREATE API FUNCTIONS (Testing)